In [17]:
# Parameters setting
Acap = 80
Bcap = 40
Ccap = 40
UPC = 1.5
EUPC = 4.5
FU = 1
NU = 0.5

In [18]:
# Read quantity matrix, i.e., the quantity to delivery between cities
import pandas as pd
file = open(r'QM.csv')
#file = open(r'New_QM2.csv')
# 读取指定目录下的csv格式的文件
file_data = pd.read_csv(file)
file_data

,origin,destination,fast_product_quantity,normal_product_quantity
0,node_01_B,node_02_B,104,2260
1,node_01_B,node_02_C,130,3633
2,node_01_B,node_02_A,56,1535
3,node_01_B,node_03_B,21,338
4,node_01_B,node_03_C,31,365
...,...,...,...,...
5963,node_27_A,node_25_C,201,335
5964,node_27_A,node_25_A,338,747
5965,node_27_A,node_26_B,87,1347
5966,node_27_A,node_26_C,223,4075


In [19]:
#读数据，每个Request的路线，fast数量，normal数量
Req = dict()

[m,n] = file_data.shape

for i in range(m):
    OD = tuple([file_data.iat[i,0],file_data.iat[i,1]])
    Req[OD] = dict()
    Req[OD]['F'] = file_data.iat[i,2]
    Req[OD]['N'] = file_data.iat[i,3]
    
    Path = []
    Path.append(OD[0])
    if OD[0][-1] != 'A':
        st = OD[0][:-1] + 'A'
        Path.append(st)
    if OD[1][-1] != 'A':
        st = OD[1][:-1] + 'A'
        Path.append(st)
    Path.append(OD[1])
    
    Req[OD]['Path'] = Path   

In [20]:
from itertools import combinations
import math
import numpy as np

Indset = list(Req.keys())

arc_set = dict()
for r in Indset:
    arc_set[r] = set()
    for arc in combinations(Req[r]['Path'],2): arc_set[r].add(arc)

Node_tt = set()
for r in Indset: Node_tt = Node_tt.union(set(Req[r]['Path']))

Node_cap = dict()
for n in Node_tt:
    if n[-1] == 'A': Node_cap[n] = Acap
    if n[-1] == 'B': Node_cap[n] = Bcap
    if n[-1] == 'C': Node_cap[n] = Ccap

Arc_tt = set()
for r in Indset:
    for arc in arc_set[r]:
        Arc_tt.add(arc)  

Arc_Xreq = dict()
for arc in Arc_tt:
    Arc_Xreq[arc] = set()
for r in Indset:
    if Req[r]['F'] > 0:
        for arc in arc_set[r]: Arc_Xreq[arc].add(r)

Arc_Yreq = dict()
for arc in Arc_tt:
    Arc_Yreq[arc] = set()
for r in Indset:
    if Req[r]['N'] > 0:
        for arc in arc_set[r]: Arc_Yreq[arc].add(r)

Out_warc = dict()
Out_varc = dict()
for n in Node_tt: 
    Out_warc[n] = set()
    Out_varc[n] = set()
for arc in Arc_tt:
    Out_warc[arc[0]].add(arc)
for arc in Arc_tt:
    Out_varc[arc[0]].add(arc)

In [21]:
import gurobipy as gp
from gurobipy import GRB
sfm = gp.Model('SFX_Model')
######################################################################################
######             Variables                                                  ########
######################################################################################        

Xind = set()
xwo = dict()
xvo = dict()
xwe = dict()
xve = dict()
for r in Indset:
    if Req[r]['F'] > 0:
        Xind.add(r)
        xwo[r] = dict()
        xvo[r] = dict()
        xwe[r] = dict()
        xve[r] = dict()
        for arc in arc_set[r]:   
            xwo[r][arc] = sfm.addVar(vtype = GRB.BINARY)
            xwe[r][arc] = sfm.addVar(vtype = GRB.BINARY)
            xvo[r][arc] = sfm.addVar(vtype = GRB.BINARY)
            xve[r][arc] = sfm.addVar(vtype = GRB.BINARY)

fwo = dict()
fvo = dict()
fwe = dict()
fve = dict()
for arc in Arc_tt:
    fwo[arc] = sfm.addVar(vtype = GRB.BINARY)
    fwe[arc] = sfm.addVar(vtype = GRB.BINARY)
    fvo[arc] = sfm.addVar(vtype = GRB.BINARY)
    fve[arc] = sfm.addVar(vtype = GRB.BINARY)

Yind = set()
ywo = dict()
yvo = dict()
ywe = dict()
yve = dict()
for r in Indset:
    if Req[r]['N'] > 0:
        Yind.add(r)
        ywo[r] = dict()
        yvo[r] = dict()
        ywe[r] = dict()
        yve[r] = dict()
        for arc in arc_set[r]:   
            ywo[r][arc] = sfm.addVar(vtype = GRB.BINARY)
            ywe[r][arc] = sfm.addVar(vtype = GRB.BINARY)
            yvo[r][arc] = sfm.addVar(vtype = GRB.BINARY)
            yve[r][arc] = sfm.addVar(vtype = GRB.BINARY)

gwo = dict()
gvo = dict()
gwe = dict()
gve = dict()
for arc in Arc_tt:
    gwo[arc] = sfm.addVar(vtype = GRB.BINARY)
    gwe[arc] = sfm.addVar(vtype = GRB.BINARY)
    gvo[arc] = sfm.addVar(vtype = GRB.BINARY)
    gve[arc] = sfm.addVar(vtype = GRB.BINARY)

In [28]:
######################################################################################
######             Constraints                                                ########
######################################################################################

for r in Xind:
    for p in Req[r]['Path']:
        in_arc = set()
        out_arc = set()
        for arc in arc_set[r]:
            if arc[1] == p: in_arc.add(arc)
            if arc[0] == p: out_arc.add(arc)

        if p == Req[r]['Path'][0]:
            sfm.addConstr(sum(xwo[r][arc] for arc in out_arc) + sum(xwe[r][arc] for arc in out_arc) +\
                          sum(xvo[r][arc] for arc in out_arc) + sum(xve[r][arc] for arc in out_arc) == 1)
        elif p == Req[r]['Path'][-1]:
            sfm.addConstr(sum(xwo[r][arc] for arc in in_arc) + sum(xwe[r][arc] for arc in in_arc) +\
                          sum(xvo[r][arc] for arc in in_arc) + sum(xve[r][arc] for arc in in_arc) == 1)
        else:
            sfm.addConstr(sum(xwo[r][arc] for arc in out_arc) + sum(xwe[r][arc] for arc in out_arc) +\
                          sum(xvo[r][arc] for arc in out_arc) + sum(xve[r][arc] for arc in out_arc) ==\
                          sum(xwo[r][arc] for arc in in_arc) + sum(xwe[r][arc] for arc in in_arc) +\
                          sum(xvo[r][arc] for arc in in_arc) + sum(xve[r][arc] for arc in in_arc))
for r in Xind:
    for arc in arc_set[r]:
        sfm.addConstr(fwo[arc] >= xwo[r][arc])
        sfm.addConstr(fwe[arc] >= xwe[r][arc])
        sfm.addConstr(fvo[arc] >= xvo[r][arc])
        sfm.addConstr(fve[arc] >= xve[r][arc])

for arc in Arc_tt:
    sfm.addConstr(fwo[arc] + fwe[arc] <= 1)
    sfm.addConstr(fvo[arc] + fve[arc] <= 1)
            
for r in Xind:
    for arc in arc_set[r]:
        if arc[-1]!= Req[r]['Path'][-1]:
            xwe[r][arc].ub=0
            xwo[r][arc].ub=0
        
for r in Yind:
    for p in Req[r]['Path']:
        in_arc = set()
        out_arc = set()
        for arc in arc_set[r]:
            if arc[1] == p: in_arc.add(arc)
            if arc[0] == p: out_arc.add(arc)
        if p == Req[r]['Path'][0]:
            sfm.addConstr(sum(ywo[r][arc] for arc in out_arc) + sum(ywe[r][arc] for arc in out_arc) +\
                          sum(yvo[r][arc] for arc in out_arc) + sum(yve[r][arc] for arc in out_arc) == 1)
        elif p == Req[r]['Path'][-1]:
            sfm.addConstr(sum(ywo[r][arc] for arc in in_arc) + sum(ywe[r][arc] for arc in in_arc) +\
                          sum(yvo[r][arc] for arc in in_arc) + sum(yve[r][arc] for arc in in_arc) == 1)
        else:
            sfm.addConstr(sum(ywo[r][arc] for arc in out_arc) + sum(ywe[r][arc] for arc in out_arc) +\
                          sum(yvo[r][arc] for arc in out_arc) + sum(yve[r][arc] for arc in out_arc) ==\
                          sum(ywo[r][arc] for arc in in_arc) + sum(ywe[r][arc] for arc in in_arc) +\
                          sum(yvo[r][arc] for arc in in_arc) + sum(yve[r][arc] for arc in in_arc))



for r in Yind:
    for arc in arc_set[r]:
        sfm.addConstr(gwo[arc] >= ywo[r][arc])
        sfm.addConstr(gwe[arc] >= ywe[r][arc])
        sfm.addConstr(gvo[arc] >= yvo[r][arc])
        sfm.addConstr(gve[arc] >= yve[r][arc])

for arc in Arc_tt:
    sfm.addConstr(gwo[arc] + gwe[arc] <= 1)
    sfm.addConstr(gvo[arc] + gve[arc] <= 1)

for r in Yind:
    for arc in arc_set[r]:
        if arc[-1]!= Req[r]['Path'][-1]:
            ywe[r][arc].ub=0
            ywo[r][arc].ub=0
            
for n in Node_tt:
    sfm.addConstr(sum(fwo[arc] for arc in Out_warc[n]) + sum(fvo[arc] for arc in Out_varc[n]) +\
                  sum(gwo[arc] for arc in Out_warc[n]) + sum(gvo[arc] for arc in Out_varc[n]) <= Node_cap[n])



######################################################################################
######         OBJECTIVE FUNCTION                                             ########
######################################################################################
sfm.setObjective(UPC*sum(Req[r]['F']*(xwo[r][arc]+xwe[r][arc]) for r in Xind for arc in arc_set[r])+\
                 (UPC+FU)*sum(Req[r]['F']*(xvo[r][arc]+xve[r][arc]) for r in Xind for arc in arc_set[r])+\
                 EUPC*sum(Req[r]['F']*xwe[r][arc] for r in Xind for arc in arc_set[r])+\
                 EUPC*sum(Req[r]['F']*xve[r][arc] for r in Xind for arc in arc_set[r])+\
                 UPC*sum(Req[r]['N']*(ywo[r][arc]+ywe[r][arc]) for r in Yind for arc in arc_set[r])+\
                 (UPC+NU)*sum(Req[r]['N']*(yvo[r][arc]+yve[r][arc]) for r in Yind for arc in arc_set[r])+\
                 EUPC*sum(Req[r]['N']*ywe[r][arc] for r in Yind for arc in arc_set[r])+\
                 EUPC*sum(Req[r]['N']*yve[r][arc] for r in Yind for arc in arc_set[r]), GRB.MINIMIZE)

In [29]:
# rel = sfm
# rel._Arc_tt = Arc_tt
# rel._InterArc_tt = InterArc_tt
# rel._fwo = fwo
# rel._fwe = fwe
# rel._fvo = fvo 
# rel._fve = fve
# rel._gwo = gwo
# rel._gwe = gwe
# rel._gvo = gvo
# rel._gve = gve
# rel._Arc_Xreq = Arc_Xreq
# rel._Arc_Yreq = Arc_Yreq
# rel._InterArc_Xreq = InterArc_Xreq
# rel._InterArc_Yreq = InterArc_Yreq
# rel._xwo = xwo
# rel._xwe = xwe
# rel._xvo = xvo
# rel._xve = xve
# rel._ywo = ywo
# rel._ywe = ywe
# rel._yvo = yvo
# rel._yve = yve

# return rel

# m = creat_Model(Req)

#sfm.Params.lazyConstraints = 1
#sfm.setParam("Cuts",0)
sfm.params.MIPGap=0.0000005
sfm.params.TimeLimit = 300 

sfm.optimize()

Set parameter MIPGap to value 5e-07
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1013896 rows, 240824 columns and 3341744 nonzeros
Model fingerprint: 0x56c9f0cc
Variable types: 0 continuous, 240824 integer (240824 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]

Loaded MIP start from previous solve with objective 2.13184e+07

Presolve removed 857216 rows and 94704 columns
Presolve time: 4.78s
Presolved: 156680 rows, 146120 columns, 523815 nonzeros
Variable types: 0 continuous, 146120 integer (146120 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2586388e+05   7.980000e+03   1.259424e+10      6s
Concurrent spin 

In [34]:
#After Processing

fwo_val = dict()
fwe_val = dict()
fvo_val = dict()
fve_val = dict()
gwo_val = dict()
gwe_val = dict()
gvo_val = dict()
gve_val = dict()

for arc in Arc_tt:
    fwo_val[arc] = fwo[arc].X
    fwe_val[arc] = fwe[arc].X
    gwo_val[arc] = gwo[arc].X
    gwe_val[arc] = gwe[arc].X
    fvo_val[arc] = fvo[arc].X
    fve_val[arc] = fve[arc].X
    gvo_val[arc] = gvo[arc].X
    gve_val[arc] = gve[arc].X

In [35]:
ID = 1
plan = dict()
for arc in Arc_tt:
    if fwo_val[arc] + fwe_val[arc] > 0:
        plan[ID] = dict()
        plan[ID]['code'] = arc[0]
        plan[ID]['plan'] = set()
        plan[ID]['cnt'] = 0
        plan[ID]['FastRequest'] = 0
        plan[ID]['NormalRequest'] = 0
        plan[ID]['extra']=fwe_val[arc]
        for r in Arc_Xreq[arc]:
            if_flow = xwo[r][arc].X + xwe[r][arc].X
            if if_flow > 0:
                plan[ID]['plan'].add(r[-1])
                plan[ID]['cnt'] = len(plan[ID]['plan'])
                plan[ID]['FastRequest'] += Req[r]['F']
        if plan[ID]['cnt'] > 0: ID += 1
            
    if gwo_val[arc] + gwe_val[arc] > 0:
        plan[ID] = dict()
        plan[ID]['code'] = arc[0]
        plan[ID]['plan'] = set()
        plan[ID]['cnt'] = 0
        plan[ID]['FastRequest'] = 0
        plan[ID]['NormalRequest'] = 0
        plan[ID]['extra']=gwe_val[arc]
        for r in Arc_Yreq[arc]:
            if_flow = ywo[r][arc].X + ywe[r][arc].X
            if if_flow > 0:
                plan[ID]['plan'].add(r[-1])
                plan[ID]['cnt'] = len(plan[ID]['plan'])
                plan[ID]['NormalRequest'] += Req[r]['N']
        if plan[ID]['cnt'] > 0: ID += 1
            
for arc in Arc_tt:
    if fvo_val[arc] + fve_val[arc] > 0:
        plan[ID] = dict()
        plan[ID]['code'] = arc[0]
        plan[ID]['plan'] = set()
        plan[ID]['cnt'] = 0
        plan[ID]['FastRequest'] = 0
        plan[ID]['NormalRequest'] = 0
        plan[ID]['extra']=fve_val[arc]
        for r in Arc_Xreq[arc]:
            if_flow = xvo[r][arc].X + xve[r][arc].X
            if if_flow > 0:
                plan[ID]['plan'].add(r[-1])
                plan[ID]['cnt'] = len(plan[ID]['plan'])
                plan[ID]['FastRequest'] += Req[r]['F']
        if plan[ID]['cnt'] > 0: ID += 1

    if gvo_val[arc] + gve_val[arc] > 0:
        plan[ID] = dict()
        plan[ID]['code'] = arc[0]
        plan[ID]['plan'] = set()
        plan[ID]['cnt'] = 0
        plan[ID]['FastRequest'] = 0
        plan[ID]['NormalRequest'] = 0
        plan[ID]['extra']=gve_val[arc]
        for r in Arc_Yreq[arc]:
            if_flow = yvo[r][arc].X + yve[r][arc].X
            if if_flow > 0:
                plan[ID]['plan'].add(r[-1])
                plan[ID]['cnt'] = len(plan[ID]['plan'])
                plan[ID]['NormalRequest'] += Req[r]['N']
        if plan[ID]['cnt'] > 0: ID += 1

IDmax = ID - 1

In [36]:
pdplan=pd.DataFrame.from_dict(plan)
pdplan=pdplan.T

plan_sort=pdplan.sort_values(by=['code','extra'])

In [38]:
import csv 

#assert len(plan_sort)==IDmax
with open('output_sort.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['ID','code','plan','cnt','fast_product_quantity','normal_product_quantity'])
    for IDind in range(len(plan_sort)):
        row = [str(IDind+1)]
        row.append(plan_sort.iloc[IDind]['code'])
        
        planlist = '、'.join(list(plan_sort.iloc[IDind]['plan']))
        row.append(planlist)
        row.append(plan_sort.iloc[IDind]['cnt'])
        row.append(plan_sort.iloc[IDind]['FastRequest'])
        row.append(plan_sort.iloc[IDind]['NormalRequest'])
        writer.writerow(row)

In [39]:
ID = 1
fullplan = dict()
for arc in Arc_tt:
    if fwo_val[arc] + fwe_val[arc] > 0:
        fullplan[ID] = dict()
        fullplan[ID]['code'] = arc[0]
        fullplan[ID]['fullplan'] = set()
        fullplan[ID]['cnt'] = 0
        fullplan[ID]['FastRequest'] = 0
        fullplan[ID]['NormalRequest'] = 0
        fullplan[ID]['extra']=fwe_val[arc]
        for r in Arc_Xreq[arc]:
            if_flow = xwo[r][arc].X + xwe[r][arc].X
            if if_flow > 0:
                fullplan[ID]['fullplan'].add(r)
                fullplan[ID]['FastRequest'] += Req[r]['F']
        fullplan[ID]['cnt']=len(set([i[-1] for i in fullplan[ID]['fullplan']]))
        if fullplan[ID]['cnt'] > 0: ID += 1
            
    if gwo_val[arc] + gwe_val[arc] > 0:
        fullplan[ID] = dict()
        fullplan[ID]['code'] = arc[0]
        fullplan[ID]['fullplan'] = set()
        fullplan[ID]['cnt'] = 0
        fullplan[ID]['FastRequest'] = 0
        fullplan[ID]['NormalRequest'] = 0
        fullplan[ID]['extra']=gwe_val[arc]
        for r in Arc_Yreq[arc]:
            if_flow = ywo[r][arc].X + ywe[r][arc].X
            if if_flow > 0:
                fullplan[ID]['fullplan'].add(r)
                fullplan[ID]['NormalRequest'] += Req[r]['N']
        fullplan[ID]['cnt']=len(set([i[-1] for i in  fullplan[ID]['fullplan']]))
        if fullplan[ID]['cnt'] > 0: ID += 1
            
for arc in Arc_tt:
    if fvo_val[arc] + fve_val[arc] > 0:
        fullplan[ID] = dict()
        fullplan[ID]['code'] = arc[0]
        fullplan[ID]['fullplan'] = set()
        fullplan[ID]['cnt'] = 0
        fullplan[ID]['FastRequest'] = 0
        fullplan[ID]['NormalRequest'] = 0
        fullplan[ID]['extra']=fve_val[arc]
        for r in Arc_Xreq[arc]:
            if_flow = xvo[r][arc].X + xve[r][arc].X
            if if_flow > 0:
                fullplan[ID]['fullplan'].add(r)
                fullplan[ID]['FastRequest'] += Req[r]['F']
        fullplan[ID]['cnt']=len(set([i[-1] for i in  fullplan[ID]['fullplan']]))
        if fullplan[ID]['cnt'] > 0: ID += 1

    if gvo_val[arc] + gve_val[arc] > 0:
        fullplan[ID] = dict()
        fullplan[ID]['code'] = arc[0]
        fullplan[ID]['fullplan'] = set()
        fullplan[ID]['cnt'] = 0
        fullplan[ID]['FastRequest'] = 0
        fullplan[ID]['NormalRequest'] = 0
        fullplan[ID]['extra']=gve_val[arc]
        for r in Arc_Yreq[arc]:
            if_flow = yvo[r][arc].X + yve[r][arc].X
            if if_flow > 0:
                fullplan[ID]['fullplan'].add(r)
                fullplan[ID]['NormalRequest'] += Req[r]['N']
        fullplan[ID]['cnt']=len(set([i[-1] for i in  fullplan[ID]['fullplan']]))
        if fullplan[ID]['cnt'] > 0: ID += 1

IDmax = ID - 1

In [158]:
pdfullplan=pd.DataFrame.from_dict(fullplan)
pdfullplan=pdfullplan.T

fullplan_sort=pdfullplan.sort_values(by=['code','extra'])


In [160]:
import csv 

#assert len(fullplan_sort)==IDmax
with open('full_output_sort.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['ID','code','plan','cnt','fast_product_quantity','normal_product_quantity'])
    for IDind in range(len(fullplan_sort)):
        row = [str(IDind+1)]
        row.append(fullplan_sort.iloc[IDind]['code'])
        
        planlist = '、'.join(f"({','.join(item)})" for item in list(fullplan_sort.iloc[IDind]['fullplan']))
        row.append(planlist)
        row.append(fullplan_sort.iloc[IDind]['cnt'])
        row.append(fullplan_sort.iloc[IDind]['FastRequest'])
        row.append(fullplan_sort.iloc[IDind]['NormalRequest'])
        writer.writerow(row)

In [1]:
4+15+30+70+90+128

337